In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import math
from gurobipy import Model, GRB, quicksum

# Section 0 Archive

In [110]:
# Rename dataframe columns
df = pd.read_csv(r"C:\Users\zwu\OneDrive - IMT Mines Albi\Documents\Data\ModelOptim\General\GeoPosition.csv")
df.head()


,Code Commune,Longitude,Latitude
0,01001,4.9209,46.1508
1,01002,5.4233,46.0072
2,01004,5.3596,45.9584
3,01005,4.9030,45.9959
4,01006,5.6011,45.7477


## Generate a map from a list of code_communes

In [111]:
demand_sites = [
    "75056",
    "13055",
    "69123",
    "31555",
    "06088",
    "44109",
    "34172",
    "67482",
    "33063",
    "59350",
    "35238",
    "51454",
    "83137",
    "42218"
]

# add arrondissments into communes list of demand
if "75056" in demand_sites:
    demand_sites+=["751"+"{:02}".format(i+1) for i in range(20)]

if "13055" in demand_sites:
    demand_sites+=["132"+"{:02}".format(i+1) for i in range(16)]

if "69123" in demand_sites:
    demand_sites+=["6938"+str(i+1) for i in range(9)]

code_communes = [
    "75056", "13055", "69123", "31555", "06088", "44109", "34172", "67482",
    "33063", "59350", "35238", "51454", "83137", "42218", "76351", "21231",
    "38185", "49007", "69266", "30189", "63113", "13001", "72181", "29019",
    "37261", "80021", "74010", "87085", "92012", "57463", "25056", "66136",
    "45234", "76540", "93066", "93048", "95018", "68224", "14118", "54395",
    "59599", "59512", "92050", "94081", "94028", "84007", "86194", "93001",
    "92004", "92025", "59183", "93005", "78646", "92026", "34032", "92063",
    "50129", "94017", "17300", "64445", "06004", "94068", "33281", "2A004",
    "06029", "44184", "93029", "93051", "68066", "92040", "95127", "62193",
    "92044", "69259", "91228", "33318", "26362", "18033", "94041", "29232"
]

code_communes_excluded = [code for code in code_communes if code.startswith('9')] + ["75056", "59599", "59350", "69266", "69259", "33063", "33281", "06004", "06029", "2A004"]

code_communes = [code for code in code_communes if code not in code_communes_excluded]

# Amazon Warehouses
# code_communes = ["45320", "26198", "71520", "59334", "80131", "38475", "77104", "45008", "77296"]


print(len(code_communes))

48


In [112]:
fig = px.scatter_mapbox(df[df["Code Commune"].isin(code_communes)], lat='Latitude', lon='Longitude', hover_name='Code Commune',
                        zoom=5, height=1000, width=1000, mapbox_style="open-street-map")
fig.update_layout()
# Show the figure
fig.show()

## Generate Demand

In [113]:
def bass_coeff(t, p=0.0019, q=0.2):
    return (1 - np.exp(-(p+q)*t))/(1+q/p*np.exp(-(p+q)*t))

file_path = r"C:\Users\zwu\OneDrive - IMT Mines Albi\Documents\Data\DemandForecasting\Vehicles\Parc automobile 2022\parc_vp_commune_2022_cleaned.csv"

rows = ['Crit\'Air 3', 'Crit\'Air 4', 'Crit\'Air 5', 'Non classé']
columns = [1, 2, 3, 4]
# initial_values = [
#     [0.25, 0.20, 0.15, 0.10],
#     [0.35, 0.30, 0.25, 0.20],
#     [0.45, 0.40, 0.35, 0.30],
#     [0.55, 0.50, 0.45, 0.40]
# ]
initial_values = [
    [0.45, 0.40, 0.35, 0.30],
    [0.55, 0.50, 0.45, 0.40],
    [0.65, 0.60, 0.55, 0.50],
    [0.75, 0.70, 0.65, 0.60]
]
percentage_table = pd.DataFrame(initial_values, index=rows, columns=columns)

def ratio_pop_den_critair(pop_den, critair):
    if pd.isna(pop_den) or pd.isna(critair):
        return None
    else:
        return percentage_table[pop_den][critair]

def calculate_num_retrofit(row):
    # Check for NaN in either "ratio" or '2022'
    if pd.isna(row["ratio"]) or pd.isna(row['2022']):
        return 0  # or some other appropriate default value
    else:
        return round(row["ratio"] * row['2022'])

pop_den = pd.read_csv("pop_den.csv")
pop_den.rename(columns={'Typo degr� de Densit�': 'Population Density'}, inplace=True)
pop_den = pop_den[['Code Commune', 'Population Density', 'coordinates']]
pop_den['lon'] = pop_den['coordinates'].str.strip('[]').str.split(',').str[0].astype(float)
pop_den['lat'] = pop_den['coordinates'].str.strip('[]').str.split(',').str[1].astype(float)    

parc_auto = pd.read_csv(file_path, encoding='ISO-8859-1')
parc_auto = parc_auto.astype({"Code commune de résidence":str})
# Take only the fleet for demand sites
parc_auto = parc_auto[parc_auto["Code commune de résidence"].isin(demand_sites)]
parc_auto[parc_auto["Code commune de résidence"].isin(["751"+"{:02}".format(i+1) for i in range(20)])]

C:\Users\zwu\AppData\Local\Temp\ipykernel_19168\3248729707.py:41: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.



,Code commune de résidence,Commune de résidence,Carburant,Crit'Air,2022
176009,75101,Paris 1er Arrondissement,Diesel,Crit'Air 3,318
176010,75101,Paris 1er Arrondissement,Diesel,Crit'Air 4,148
176011,75101,Paris 1er Arrondissement,Diesel,Crit'Air 5,29
176012,75101,Paris 1er Arrondissement,Diesel,Non classé,26
176013,75101,Paris 1er Arrondissement,Essence,Crit'Air 3,407
...,...,...,...,...,...
176124,75120,Paris 20e Arrondissement,Diesel,Crit'Air 4,1724
176125,75120,Paris 20e Arrondissement,Diesel,Crit'Air 5,341
176126,75120,Paris 20e Arrondissement,Diesel,Non classé,210
176127,75120,Paris 20e Arrondissement,Essence,Crit'Air 3,3854


In [114]:

def replace_arrondissements(code):
    mappings = {
    (69381, 69389): '69123',
    (75101, 75120): '75056', 
    (13201, 13216): '13055'
    }
    # Iterate through the mapping
    for original_range, new_value in mappings.items():
        start, end = original_range
        # Generate the list of codes in the current range to be replaced
        codes_to_replace = [str(i) for i in range(start, end + 1)]
        if code in codes_to_replace:
            return new_value
        else: return code

mapping_name = {
    '69123':'Lyon',
    '75056':'Paris',
    '13055':'Marseille'}

parc_auto["Code commune de résidence"] = parc_auto["Code commune de résidence"].apply(lambda x:replace_arrondissements(x))
parc_auto.loc[parc_auto["Code commune de résidence"].isin(mapping_name.keys()), 'Commune de résidence'] = parc_auto['Code commune de résidence'].map(mapping_name)

In [115]:
parc_auto = parc_auto.groupby(["Code commune de résidence", "Commune de résidence", "Carburant", "Crit\'Air"]).sum().reset_index()

In [116]:
parc_auto_pop_den = pd.merge(parc_auto, pop_den, left_on='Code commune de résidence', right_on='Code Commune', how='left')[['Code commune de résidence', 'Commune de résidence', 'lon', 'lat', 'Carburant', 'Crit\'Air', '2022', 'Population Density']]
parc_auto_pop_den["ratio"] = parc_auto_pop_den.apply(lambda row: ratio_pop_den_critair(row['Population Density'], row['Crit\'Air']), axis=1)
parc_auto_pop_den['num_retrofit'] = parc_auto_pop_den.apply(calculate_num_retrofit, axis=1)
parc_auto_pop_den = parc_auto_pop_den[parc_auto_pop_den['num_retrofit'] != 0]
parc_auto_pop_den.rename({"2022":"total"}, axis=1, inplace=True)
parc_retrofit_total = parc_auto_pop_den[["Code commune de résidence", "Commune de résidence", "num_retrofit"]].groupby(by=["Code commune de résidence", "Commune de résidence"]).sum().reset_index()
parc_retrofit_total

,Code commune de résidence,Commune de résidence,num_retrofit
0,06088,Nice,23309
1,13055,Marseille,57689
2,31555,Toulouse,34790
3,33063,Bordeaux,16098
4,34172,Montpellier,21877
5,35238,Rennes,15289
6,42218,Saint-Étienne,14110
7,44109,Nantes,20979
8,51454,Reims,13064
9,59350,Lille,13584


In [117]:
planning_horizon_years = 5
coeff_df = pd.DataFrame(columns=["Year", "Coefficient"])
coeff_df["Year"] = range(1, planning_horizon_years+1)
coeff_df["Coefficient"]=coeff_df["Year"].map(lambda x: bass_coeff(x))
coeff_df

,Year,Coefficient
0,1,0.002101
1,2,0.004660
2,3,0.007774
3,4,0.011558
4,5,0.016149


In [118]:
parc_retrofit_yearly = parc_retrofit_total.merge(coeff_df, how="cross")
parc_retrofit_yearly["num_retrofit"] = round(parc_retrofit_yearly["num_retrofit"] * parc_retrofit_yearly["Coefficient"])
parc_retrofit_yearly

,Code commune de résidence,Commune de résidence,num_retrofit,Year,Coefficient
0,06088,Nice,49.0,1,0.002101
1,06088,Nice,109.0,2,0.004660
2,06088,Nice,181.0,3,0.007774
3,06088,Nice,269.0,4,0.011558
4,06088,Nice,376.0,5,0.016149
...,...,...,...,...,...
65,83137,Toulon,30.0,1,0.002101
66,83137,Toulon,67.0,2,0.004660
67,83137,Toulon,111.0,3,0.007774
68,83137,Toulon,165.0,4,0.011558


In [119]:
fig = px.bar(parc_retrofit_yearly, x="Year", y="num_retrofit", color="Commune de résidence", barmode='stack')
fig.show()

In [120]:
parc_retrofit_yearly.sum()

Code commune de résidence    0608806088060880608806088130551305513055130551...
Commune de résidence         NiceNiceNiceNiceNiceMarseilleMarseilleMarseill...
num_retrofit                                                           15173.0
Year                                                                       210
Coefficient                                                           0.591382
dtype: object

In [121]:
year = 1
df_show=parc_retrofit_yearly[parc_retrofit_yearly["Year"]==year]
df_show = pd.merge(df, df_show, left_on="Code Commune", right_on="Code commune de résidence", how="right")
fig = px.scatter_mapbox(df_show, lat='Latitude', lon='Longitude', hover_name='Code Commune', size="num_retrofit",
                        zoom=3, height=500, width=500, mapbox_style="open-street-map")

# Show the figure
fig.show()

# Section 1 Create test dataset

In [2]:
P=["p1","p2"]
M={"Paris":[48.8012, 2.1303], "Lyon":[45.7676, 4.8350], "Toulouse":[43.6043, 1.4437]}
R={"Paris":[48.8012, 2.1F303], "Lyon":[45.7676, 4.8350], "Toulouse":[43.6043, 1.4437], "Clermont-Ferrand":[45.7831, 3.0822], "Orleans":[47.9025, 1.9089]}
L={"Paris":[48.8012, 2.1303],"Clermont-Ferrand":[45.7831, 3.0822]}
F={"Clermont-Ferrand":[45.7831, 3.0822], "Orleans":[47.9025, 1.9089]}
V={"Clermont-Ferrand":[45.7831, 3.0822], "Orleans":[47.9025, 1.9089]}

wru = 300
wrpd = 1300
weolpd = 1300
wrtp = 220
wrp = 180
tc = 0.05

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d

distance_matrix = pd.DataFrame(index=R.keys(), columns=R.keys())
for i in R.keys():
    for j in R.keys():
        distance_matrix.loc[i, j] = haversine(R[i][0], R[i][1], R[j][0], R[j][1])

installation_cost = pd.DataFrame(columns=["Type", "Location", "Installation Cost"])
installation_cost["Type"] = ["Factory" for f in F] + ["Logistics Node" for l in L] + ["Retrofit Center" for r in R] + ["Recovery Center" for v in V]
installation_cost["Location"] = [f for f in F.keys()]+ [l for l in L.keys()] + [r for r in R.keys()] + [v for v in V.keys()]

location_multipliers = {
    'Paris': 2,
    'Lyon': 1.6,
    'Toulouse': 1.5,
    'Orleans': 1.3,
    'Clermont-Ferrand': 1.0
}

type_multipliers = {
    'Factory': 3,
    'Logistics Node': 1.5,
    'Retrofit Center': 2.5,
    'Recovery Center': 1
}
base_installation_cost = 200000
base_capacity_cost = 100000
installation_cost["Installation Cost"] = installation_cost.apply(lambda row: base_installation_cost * location_multipliers[row["Location"]] * type_multipliers[row["Type"]], axis=1)

dm_mp = pd.DataFrame(index=M.keys(), columns=P)
dm_mp["p1"] = [300000, 180000, 160000]
dm_mp["p2"] = [200000, 150000, 130000]
dmeol_mp = pd.DataFrame(index=M.keys(), columns=P)
dmeol_mp["p1"] = [3000, 1800, 1600]
dmeol_mp["p2"] = [2000, 1500, 1300]

big_M = 1000000

In [3]:
# Visualize the input data
# Put factories on a map


df_factory = pd.DataFrame(F).T
df_factory.columns = ["Latitude", "Longitude"]
df_factory["Type"] = "Factory"
df_logistics = pd.DataFrame(L).T
df_logistics.columns = ["Latitude", "Longitude"]
df_logistics["Type"] = "Logistics Node"
df_retail = pd.DataFrame(R).T
df_retail.columns = ["Latitude", "Longitude"]
df_retail["Type"] = "Retrofit Center"
df_recovery = pd.DataFrame(V).T
df_recovery.columns = ["Latitude", "Longitude"]
df_recovery["Type"] = "Recovery Center"
df_market = pd.DataFrame(M).T
df_market.columns = ["Latitude", "Longitude"]
df_market["Type"] = "Market"
df = pd.concat([df_factory, df_logistics, df_retail, df_recovery, df_market])
df

,Latitude,Longitude,Type
Clermont-Ferrand,45.7831,3.0822,Factory
Orleans,47.9025,1.9089,Factory
Paris,48.8012,2.1303,Logistics Node
Clermont-Ferrand,45.7831,3.0822,Logistics Node
Paris,48.8012,2.1303,Retrofit Center
Lyon,45.7676,4.8350,Retrofit Center
Toulouse,43.6043,1.4437,Retrofit Center
Clermont-Ferrand,45.7831,3.0822,Retrofit Center
Orleans,47.9025,1.9089,Retrofit Center
Clermont-Ferrand,45.7831,3.0822,Recovery Center


In [4]:
type_to_icon = {
    'Factory': 'industrial',
    'Logistics Node': 'warehouse',
    'Retrofit Center': 'car-repair',
    'Recovery Center': 'recycling',
    'Market': 'shop'
}

# Initialize a Plotly figure
fig = go.Figure()

# Loop through each facility type and add a scattermapbox trace for each
for facility_type, group_df in df.groupby('Type'):
    fig.add_trace(go.Scattermapbox(
        lat=group_df['Latitude'],
        lon=group_df['Longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=9,
            symbol=type_to_icon[facility_type]
        ),
        name=facility_type
    ))

# Update layout with Mapbox access token and style
fig.update_layout(
    mapbox_style="light",  # or use "open-street-map" if you don't have a Mapbox token
    mapbox_zoom=5,
    mapbox_center={"lat": df['Latitude'].mean(), "lon": df['Longitude'].mean()}
)

# Show the figure
fig.show()

In [126]:
model = Model("SupplyChainDesign1")

# Variables
x_f = model.addVars(F, vtype=GRB.BINARY, name="x_f")
x_l = model.addVars(L, vtype=GRB.BINARY, name="x_l")
x_r = model.addVars(R, vtype=GRB.BINARY, name="x_r")
x_v = model.addVars(V, vtype=GRB.BINARY, name="x_v")
q_flp = model.addVars(F, L, P, vtype=GRB.CONTINUOUS, name="q_flp")
q_lrp = model.addVars(L, R, P, vtype=GRB.CONTINUOUS, name="q_lrp")
q_rmp = model.addVars(R, M, P, vtype=GRB.CONTINUOUS, name="q_rmp")
q_mvp = model.addVars(M, V, P, vtype=GRB.CONTINUOUS, name="q_mvp")
q_rvp = model.addVars(R, V, P, vtype=GRB.CONTINUOUS, name="q_rvp")
q_vlp = model.addVars(V, L, P, vtype=GRB.CONTINUOUS, name="q_vlp")
q_vfp = model.addVars(V, F, P, vtype=GRB.CONTINUOUS, name="q_vfp")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-26


MIN Total Cost = Transportation Cost (TC) + Installation Cost (IC) 
Transportation Cost = $tc \times \left[ \sum_{f \in F} \sum_{l \in L} \sum_{p \in P} q_{flp} \cdot w{ru} \cdot d_{fl} + \sum_{l \in L} \sum_{r \in R} \sum_{p \in P} q_{lrp} \cdot w{ru} \cdot d_{lr} +  \sum_{r \in R} \sum_{m \in M} \sum_{p \in P} q_{rmp} \cdot w{rpd} \cdot d_{rm} +  \sum_{r \in R} \sum_{v \in V} \sum_{p \in P} q_{rvp} \cdot w{rtp} \cdot d_{rv} + \sum_{v \in V} \sum_{l \in L} \sum_{p \in P} q_{vlp} \cdot w{ru} \cdot d_{vl} + \sum_{v \in V} \sum_{f \in F} \sum_{p \in P} q_{vfp} \cdot w{rp} \cdot d_{vf} +  \sum_{m \in M} \sum_{v \in V} \sum_{p \in P} q_{mvp} \cdot w_{eolpd} \cdot d_{mv} \right]$

In [127]:
# Objective
transportation = quicksum(tc * q_flp[f, l, p] * wru * distance_matrix.loc[f, l] for f in F for l in L for p in P) + quicksum(tc * q_lrp[l, r, p] * wru * distance_matrix.loc[l, r] for l in L for r in R for p in P) 
transportation += quicksum(tc * q_rmp[r, m, p] * wrpd * distance_matrix.loc[r, m] for r in R for m in M for p in P) 
transportation += quicksum(tc * q_mvp[m, v, p] * weolpd * distance_matrix.loc[m, v] for m in M for v in V for p in P) 
transportation += quicksum(tc * q_rvp[r, v, p] * wrtp * distance_matrix.loc[r, v] for r in R for v in V for p in P) 
transportation += quicksum(tc * q_vlp[v, l, p] * wru * distance_matrix.loc[v, l] for v in V for l in L for p in P) 
transportation += quicksum(tc * q_vfp[v, f, p] * wrp * distance_matrix.loc[v, f] for v in V for f in F for p in P)


installation = quicksum(installation_cost[(installation_cost.Type=="Factory") & (installation_cost.Location==f)]["Installation Cost"].values[0] * x_f[f] for f in F) 
installation += quicksum(installation_cost[(installation_cost.Type=="Logistics Node") & (installation_cost.Location==l)]["Installation Cost"].values[0] * x_l[l] for l in L) 
installation += quicksum(installation_cost[(installation_cost.Type=="Retrofit Center") & (installation_cost.Location==r)]["Installation Cost"].values[0] * x_r[r] for r in R) 
installation += quicksum(installation_cost[(installation_cost.Type=="Recovery Center") & (installation_cost.Location==v)]["Installation Cost"].values[0] * x_v[v] for v in V)

model.setObjective(transportation + installation, GRB.MINIMIZE)

In [128]:
# Constraints
# Demand fulfillment constraints
for m_seg in M:
    for p in P:
        model.addConstr(quicksum(q_rmp[r, m_seg, p] for r in R) == dm_mp.loc[m_seg, p])
        model.addConstr(quicksum(q_mvp[m_seg, v, p] for v in V) == dmeol_mp.loc[m_seg, p])

# Facility operation constraints
for f in F:
    for p in P:
        for l in L:
            model.addConstr(q_flp[f, l, p] <= big_M * x_f[f])
            model.addConstr(q_flp[f, l, p] <= big_M * x_l[l])
for l in L:
    for p in P:
        for r in R:
            model.addConstr(q_lrp[l, r, p] <= big_M * x_l[l])
            model.addConstr(q_lrp[l, r, p] <= big_M * x_r[r])
for r in R:
    for p in P:
        for m in M:
            model.addConstr(q_rmp[r, m, p] <= big_M * x_r[r])
for r in R:
    for p in P:
        for v in V:
            model.addConstr(q_rvp[r, v, p] <= big_M * x_r[r])
            model.addConstr(q_rvp[r, v, p] <= big_M * x_v[v])
for v in V:
    for p in P:
        for l in L:
            model.addConstr(q_vlp[v, l, p] <= big_M * x_v[v])
            model.addConstr(q_vlp[v, l, p] <= big_M * x_l[l])
for v in V:
    for p in P:
        for f in F:
            model.addConstr(q_vfp[v, f, p] <= big_M * x_v[v])
            model.addConstr(q_vfp[v, f, p] <= big_M * x_f[f])
for m in M:
    for p in P:
        for v in V:
            model.addConstr(q_mvp[m, v, p] <= big_M * x_v[v])

# Flow conservation constraints
for f in F:
    for p in P:
        model.addConstr(quicksum(q_flp[f, l, p] for l in L) >= quicksum(q_vfp[v, f, p] for v in V))
for l in L:
    for p in P:
        model.addConstr(quicksum(q_flp[f, l, p] for f in F) + quicksum(q_vlp[v, l, p] for v in V) == quicksum(q_lrp[l, r, p] for r in R))
for r in R:
    for p in P:
        model.addConstr(quicksum(q_lrp[l, r, p] for l in L) == quicksum(q_rmp[r, m, p] for m in M))
for r in R:
    for p in P:
        model.addConstr(quicksum(q_rmp[r, m, p] for m in M) == quicksum(q_rvp[r, v, p] for v in V))
for v in V:
    for p in P:
        model.addConstr(quicksum(q_vlp[v, l, p] for l in L) + quicksum(q_vfp[v, f, p] for f in F) <= quicksum(q_mvp[m, v, p] for m in M))

In [129]:
model.optimize()
if model.status == GRB.OPTIMAL:
    print("Optimal solution found with total cost", model.objVal)
    for v in model.getVars():
        if v.x > 0:
            print(v.varName, v.x)


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10850H CPU @ 2.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 214 rows, 117 columns and 562 nonzeros
Model fingerprint: 0x05f0aeb3
Variable types: 106 continuous, 11 integer (11 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+03, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 3e+05]
Found heuristic solution: objective 2.895407e+10
Presolve removed 26 rows and 14 columns
Presolve time: 0.01s
Presolved: 188 rows, 103 columns, 494 nonzeros
Variable types: 92 continuous, 11 integer (11 binary)

Root relaxation: objective 4.671761e+09, 31 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4.6718e